In [ ]:
from pathlib import Path

import numpy as np
import nibabel as nib


array_size = 256
slice_num = 500

pixel_width_mm = 3.125
slice_width_mm = 3.125 
spacing = (pixel_width_mm, pixel_width_mm, slice_width_mm) # (x, y, z)

phatom_root_dir = Path.cwd().resolve().parent
xcat_dir = phatom_root_dir / "XCAT"
raw_root_dir = phatom_root_dir / "output_raw"
assert xcat_dir.exists() and xcat_dir.is_dir()

raw_prefix = "output_test"
raw_dir = raw_root_dir / raw_prefix

nii_dir = phatom_root_dir / "output_nii" / raw_prefix
nii_dir.mkdir(parents=True, exist_ok=True)


In [ ]:
frame = 1
raw_file = raw_dir / f"{raw_prefix}_atn_{frame}.bin"

with open(raw_file, "rb") as f:
    raw_data = np.fromfile(f, dtype=np.float32)

expected_shape = (slice_num, array_size, array_size) # 原始轴顺序为(z, y, x)
expected_size = np.prod(expected_shape)

assert raw_data.size == expected_size
data_3d = raw_data.reshape(expected_shape) # 原始轴顺序为(z, y, x)
data_3d = data_3d.transpose(2, 1, 0) # 转换为(x, y, z)

print(data_3d.shape)

affine = np.eye(4)
affine[(0,1,2), (0,1,2)] = spacing
nii_img = nib.Nifti1Image(data_3d, affine)
nib.save(nii_img, nii_dir / f"{raw_prefix}_atn_{frame}.nii.gz")

(256, 256, 500)
